Import Library yang dibutuhkan

In [1]:
# Import library
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import zipfile
import os

Membaca data pada format csv

In [3]:
book = pd.read_csv('dataset/Books.csv')
ratings = pd.read_csv('dataset/Ratings.csv')
user = pd.read_csv('dataset/Users.csv')

C:\Users\raiha\AppData\Local\Temp\ipykernel_24604\910916565.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('dataset/Books.csv')


# Univariate Exploratory Data Analysis

## Variable buku

Membaca Informasi DataFrame buku

In [4]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


Mengambil 5 data awal DataFrame buku

In [5]:
book.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


## Variable ratings

Membaca Informasi DataFrame ratings

In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


Mengambil 5 DataFrame awal ratings

In [7]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Melihat statistik deskriptif ratings

In [8]:
ratings.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


Menhitung Jumlah unik dari User ID dan Buku, dan total jumlah data rating dari DataFrame ratings

In [9]:
print('Jumlah userID: ', len(ratings['User-ID'].unique()))
print('Jumlah Buku: ', len(ratings.ISBN.unique()))
print('Jumlah data rating: ', len(ratings))

Jumlah userID:  105283
Jumlah Buku:  340556
Jumlah data rating:  1149780


## Variable User

Membaca Informasi DataFrame user

In [10]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


Mengambil 5 DataFrame awal user

In [11]:
user.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


Melihat statistik deskriptif ratings

In [12]:
user.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


# Data Preprocessing

Menggabungkan dua DataFrame ratings dan book bedasarkan kolom ISBN

In [13]:
merge1 = pd.merge(ratings, book, on='ISBN')
merge1.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


Menggabungkan DataFrame merge1 dan user bedasarkan kolom User-ID

In [14]:
merge2 = pd.merge(merge1, user, on='User-ID')
merge2.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa",NaN
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"cincinnati, ohio, usa",23.0
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,"cincinnati, ohio, usa",23.0
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,http://images.amazon.com/images/P/0679745580.0...,http://images.amazon.com/images/P/0679745580.0...,http://images.amazon.com/images/P/0679745580.0...,"cincinnati, ohio, usa",23.0
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,1996,HarperCollins,http://images.amazon.com/images/P/0060173289.0...,http://images.amazon.com/images/P/0060173289.0...,http://images.amazon.com/images/P/0060173289.0...,"cincinnati, ohio, usa",23.0


## Data Preparation

Membuang kolom yang tidak dipakai pada DataFrame merge2

In [15]:
merge2.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'Year-Of-Publication', 'Location', 'Age'], axis=1, inplace=True)
merge2.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,Ballantine Books
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,Tor Books
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,Vintage
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,HarperCollins


Memeriksa Null pada DataFrame merge2

In [16]:
merge2.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
Book-Title     0
Book-Author    2
Publisher      2
dtype: int64

Membuang data yang null pada DataFrame merge2

In [17]:
merge2.dropna(inplace=True)

Membuang DataFrame yang Book-Ratingnya kurang dari 0 dan memeriksa statistik nya

In [18]:
# Membuang DataFrame yang Book-Rating kurang dari 0
merge2=merge2[merge2["Book-Rating"]>0]

# Memeriksa statistik deskriptif Book-Rating
merge2["Book-Rating"].describe()

count    383838.000000
mean          7.626686
std           1.841342
min           1.000000
25%           7.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: Book-Rating, dtype: float64

Menghitung jumlah User Unik

In [19]:
merge2["User-ID"].nunique()

68091

Menghitung jumlah ISBN Unik

In [20]:
merge2["ISBN"].nunique()

149832

Menghitung jumlah Judul buku Unik

In [21]:
merge2["Book-Title"].nunique()

135564

Membuat salinan dari DataFrame merge2 dan menyimpannya dalam variabel preparation. Kemudian, preparation.sort_values('ISBN') mengurutkan DataFrame berdasarkan kolom ISBN.

In [22]:
preparation = merge2
preparation.sort_values('ISBN')

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Publisher
508164,171118,0000913154,8,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator),Simon &amp; Schuster
602563,23902,0001046438,9,Liar,Stephen Fry,Harpercollins Uk
602564,23902,000104687X,6,"T.S. Eliot Reading \The Wasteland\"" and Other ...",T.S. Eliot,HarperCollins Publishers
602565,23902,0001047213,9,The Fighting Man,Gerald Seymour,HarperCollins Publishers
523321,11944,0001047973,9,Brave New World,Aldous Huxley,Trafalgar Square Publishing
...,...,...,...,...,...,...
574211,192093,B0001FZGRQ,9,The Clan of the Cave Bear,Jean M. Auel,Crown Publishing Group
398626,24194,B0001GMSV2,8,Find Me,Rosie O'Donnell,Warner Books
477894,179772,B0001GMSV2,8,Find Me,Rosie O'Donnell,Warner Books
88396,148258,B0001I1KOG,10,New York Public Library Literature Companion,New York Public Library,Free Press


Kode dibawah berfungsi untuk membuat kolom baru bernama Features di DataFrame preparation. Kolom ini adalah hasil penggabungan kolom Book-Title, Book-Author, dan Publisher yang dipisahkan oleh koma.

In [23]:
# Membuat kolom 'Features' dengan menggabungkan kolom 'Book-Title', 'Book-Author', dan 'Publisher' dipisahkan oleh koma
preparation['Features'] = preparation[['Book-Title', 'Book-Author', 'Publisher']].agg(', '.join, axis=1)

In [24]:
preparation.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Publisher,Features
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,"Flesh Tones: A Novel, M. J. Rose, Ballantine B..."
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,Tor Books,"Ender's Game (Ender Wiggins Saga (Paperback)),..."
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,Vintage,"In Cold Blood (Vintage International), TRUMAN ..."
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,HarperCollins,Divine Secrets of the Ya-Ya Sisterhood : A Nov...
5,2313,0385482388,5,The Mistress of Spices,Chitra Banerjee Divakaruni,Anchor Books/Doubleday,"The Mistress of Spices, Chitra Banerjee Divaka..."


Menghapus row yang valuenya NaN pada kolom Features

In [25]:
preparation.dropna(subset=['Features'], inplace=True)

Membuat nilai pada kolom Features menjadi huruf kecil semua

In [26]:
preparation['Features'] = preparation['Features'].astype(str).str.lower()
preparation

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Publisher,Features
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,"flesh tones: a novel, m. j. rose, ballantine b..."
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,Tor Books,"ender's game (ender wiggins saga (paperback)),..."
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,Vintage,"in cold blood (vintage international), truman ..."
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,HarperCollins,divine secrets of the ya-ya sisterhood : a nov...
5,2313,0385482388,5,The Mistress of Spices,Chitra Banerjee Divakaruni,Anchor Books/Doubleday,"the mistress of spices, chitra banerjee divaka..."
...,...,...,...,...,...,...,...
1031130,276442,2264032960,6,L'Apprenti du diable,Ellis Peters,Editions 10/18,"l'apprenti du diable, ellis peters, editions 1..."
1031131,276442,2862749796,7,Le Huit,Katherine Neville,Le Cherche Midi,"le huit, katherine neville, le cherche midi"
1031132,276618,3788097000,5,Ludwig Marum: Briefe aus dem Konzentrationslag...,Ludwig Marum,C.F. MÃ¼ller,ludwig marum: briefe aus dem konzentrationslag...
1031134,276647,0689822294,10,Heaven (Coretta Scott King Author Award Winner),Angela Johnson,Simon &amp; Schuster Children's Publishing,heaven (coretta scott king author award winner...


Memeriksa Nilai duplicate yang ada di kolom Book-Title

In [27]:
preparation['Book-Title'].value_counts()

Book-Title
The Lovely Bones: A Novel     707
Wild Animus                   581
The Da Vinci Code             494
The Secret Life of Bees       406
The Nanny Diaries: A Novel    393
                             ... 
The Other Great Depression      1
Death in Summer                 1
Walk into the Night             1
Christopher and Alexandra       1
ROBOT RACE (MICRO ADV 6)        1
Name: count, Length: 135564, dtype: int64

Menghapus data duplicate yang ada dikolom Book-Title

In [28]:
preparation.drop_duplicates(subset='Book-Title', keep='first', inplace=True)

Mengambil 5 data awal data preparation

In [29]:
preparation.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Publisher,Features
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,"flesh tones: a novel, m. j. rose, ballantine b..."
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,Tor Books,"ender's game (ender wiggins saga (paperback)),..."
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,Vintage,"in cold blood (vintage international), truman ..."
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,HarperCollins,divine secrets of the ya-ya sisterhood : a nov...
5,2313,0385482388,5,The Mistress of Spices,Chitra Banerjee Divakaruni,Anchor Books/Doubleday,"the mistress of spices, chitra banerjee divaka..."


mengonversi kolom ISBN, Book-Title, dan Features dari DataFrame preparation menjadi daftar (list) dan mencetak length masing-masing daftar.

In [30]:
ISBN = preparation['ISBN'].tolist()
Book_Title = preparation['Book-Title'].tolist()
Book_Features = preparation['Features'].tolist()

print(len(ISBN))
print(len(Book_Title))
print(len(Book_Features))

135564
135564
135564


Membatasi jumlah data dalam daftar ISBN, Book_Title, dan Book_Features menjadi maksimal 10.000 data

In [31]:
ISBN = ISBN[:10000]
Book_Title = Book_Title[:10000]
Book_Features = Book_Features[:10000]

print(len(ISBN))
print(len(Book_Title))
print(len(Book_Features))


10000
10000
10000


Membuat dictionary untuk menentukan pasangan key-value pada data ISBN, Book_Title, dan Book_Title

In [32]:
book_new = pd.DataFrame({
    'ISBN': ISBN,
    'Book_Title': Book_Title,
    'Book_Features': Book_Features
})
book_new

,ISBN,Book_Title,Book_Features
0,034545104X,Flesh Tones: A Novel,"flesh tones: a novel, m. j. rose, ballantine b..."
1,0812533550,Ender's Game (Ender Wiggins Saga (Paperback)),"ender's game (ender wiggins saga (paperback)),..."
2,0679745580,In Cold Blood (Vintage International),"in cold blood (vintage international), truman ..."
3,0060173289,Divine Secrets of the Ya-Ya Sisterhood : A Novel,divine secrets of the ya-ya sisterhood : a nov...
4,0385482388,The Mistress of Spices,"the mistress of spices, chitra banerjee divaka..."
...,...,...,...
9995,0060183020,The Late Night Muse,"the late night muse, bette pesetsky, harpercol..."
9996,0060183772,A God in Ruins,"a god in ruins, leon uris, harpercollins publi..."
9997,0060185074,The Vendetta Defense,"the vendetta defense, lisa scottoline, harperc..."
9998,0060186453,Monica: From Fear to Victory,"monica: from fear to victory, monica seles, ha..."


Mengambil 5 sample acak dari DataFrame data

In [33]:
data = book_new
data.sample(5)

,ISBN,Book_Title,Book_Features
4003,031232135X,Flat Crazy : A Blanco County Mystery (Game War...,flat crazy : a blanco county mystery (game war...
9131,0064407314,Monster (rpkg),"monster (rpkg), walter dean myers, amistad"
9688,0425120015,Playmates,"playmates, robert b. parker, berkley publishin..."
8495,0345352661,Guardians of the West (Book 1 of the Malloreon),guardians of the west (book 1 of the malloreon...
646,0743453069,Charming the Highlander (Highlander Trilogy),"charming the highlander (highlander trilogy), ..."


Mengekstraksi fitur dari teks dalam kolom Book_Features dan mendapatkan nama-nama fitur (kata-kata) yang dihasilkan oleh vektorisasi

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
tf.fit(data['Book_Features'])
tf.get_feature_names_out()

array(['00', '000', '003', ..., 'â¼skind', 'â¼sterer', 'â¼ten'],
      dtype=object)

Melakukan transformasi teks pada kolom Book_Features menggunakan TfidfVectorizer dan kemudian memeriksa bentuk (shape) dari matriks TF-IDF yang dihasilkan

In [35]:
tfidf_matrix = tf.fit_transform(data['Book_Features'])
tfidf_matrix.shape

(10000, 13945)

menghasilkan vektor TF-IDF dalam bentuk matriks, menggunakan fungsi todense()

In [36]:
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

Membuat DataFrame dari matriks TF-IDF, dengan kolom sebagai nama fitur (kata-kata) dan baris sebagai judul buku dari DataFrame data. Kemudian, akan menampilkan sampel acak dari 10 baris dan 22 kolom

In [37]:
pd.DataFrame(
    tfidf_matrix.todense(),
    columns=tf.get_feature_names_out(),
    index=data['Book_Title']
).sample(22, axis=1).sample(10, axis=0)

,siamese,ridley,cereal,yule,porch,strokes,gordon,eslick,interplanetary,boundary,...,mccarthy,droppings,992,auf,beekeeper,professor,microcosms,liddy,mayo,glitters
Book_Title,,,,,,,,,,,,,,,,,,,,,
Drums Along The Jacks Fork,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Little House On the Prairie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Their Wildest Dreams,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Time Stand Still: A Darren Camponi Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Otherwise Engaged,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hidden Prey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Love Me Again (Zebra Historical Romance),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Misconception,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Modelling Content Based Filter

Menghitung cosine antara dokumen dalam matriks TF-IDF menggunakan cosine_similarity dari sklearn.metrics.pairwise. Kemiripan cosine mengukur seberapa mirip dua dokumen berdasarkan fitur teks mereka.

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1.        , 0.01288853, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01288853, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.06335167,
        0.08322551],
       [0.        , 0.        , 0.        , ..., 0.06335167, 1.        ,
        0.06374704],
       [0.        , 0.        , 0.        , ..., 0.08322551, 0.06374704,
        1.        ]])

Membuat DataFrame dari matriks cosine similarity dan menampilkan sampel acak dari 10 baris dan 5 kolom dari DataFrame tersebut

---



In [39]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['Book_Title'], columns=data['Book_Title'])
print('Shape:', cosine_sim_df.shape)

cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (10000, 10000)


Book_Title,The Getaway Special,A Love to Cherish,Generation Mama. ... und die Welt steht Kopf.,Dhampir,My Wicked Earl (Avon Romantic Treasure)
Book_Title,,,,,
Pain-Free Arthritis: A 7-Step Plan for Feeling Better Again,0.010451,0.000000,0.0,0.0,0.000000
Prince of Lost Places,0.000000,0.000000,0.0,0.0,0.000000
Abide With Me,0.016635,0.000000,0.0,0.0,0.000000
Sun Valley,0.000000,0.000000,0.0,0.0,0.000000
Is There Life After Housework,0.012421,0.000000,0.0,0.0,0.000000
Attack/Killer Tomatoes PT N Mk,0.024696,0.000000,0.0,0.0,0.000000
The Wedding Wager,0.014354,0.000000,0.0,0.0,0.113362
Cause of Fear,0.018067,0.000000,0.0,0.0,0.000000
"Star Light, Star Bright",0.017108,0.000000,0.0,0.0,0.000000


Memberikan rekomendasi buku berdasarkan kemiripan dengan buku yang diberikan.

In [40]:
def book_recommendations(book_title, similarity_data=cosine_sim_df, items=data[['ISBN', 'Book_Title', 'Book_Features']], k=5):
    """
    Memberikan rekomendasi buku berdasarkan kemiripan dengan buku yang diberikan.

    Parameter:
    ---
    book_title : str
        Judul buku yang digunakan untuk mencari rekomendasi.
    similarity_data : pd.DataFrame
        DataFrame yang berisi kemiripan kosinus antara buku-buku.
    items : pd.DataFrame
        DataFrame yang berisi informasi buku (ISBN, judul buku).
    k : int
        Jumlah rekomendasi yang diinginkan.
    ---

    Return:
    pd.DataFrame
        DataFrame yang berisi k buku yang paling mirip dengan buku yang diberikan.
    """

    # Mengambil indeks dari k kemiripan tertinggi
    index = similarity_data.loc[:,book_title].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mendapatkan judul buku yang paling mirip
    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    # Menghapus buku yang dicari dari hasil rekomendasi
    closest = closest.drop(book_title, errors='ignore')

    # Menggabungkan dengan informasi buku dan mengembalikan k rekomendasi teratas
    return pd.DataFrame(closest).merge(items).head(k)


Menampilkan rekomendasi buku

In [56]:
book_recommendations("Harry Potter and the Sorcerer's Stone (Book 1)")

,Book_Title,ISBN,Book_Features
0,Harry Potter and the Sorcerer's Stone (Harry P...,059035342X,harry potter and the sorcerer's stone (harry p...
1,"Harry Potter and the Sorcerer's Stone (Book 1,...",0807281751,"harry potter and the sorcerer's stone (book 1,..."
2,Harry Potter and the Chamber of Secrets (Book 2),0439064872,harry potter and the chamber of secrets (book ...
3,Harry Potter and the Goblet of Fire (Book 4),0439139597,"harry potter and the goblet of fire (book 4), ..."
4,The Sorcerer's Companion: A Guide to the Magic...,0767908473,the sorcerer's companion: a guide to the magic...


## Evaluasi

Menggunakan nilai cosine similarity langsung sebagai kategori dalam ground truth.

In [48]:
# Menggunakan nilai cosine similarity langsung sebagai kategori
ground_truth = cosine_sim

# Menampilkan beberapa nilai pada matriks ground truth
ground_truth_df = pd.DataFrame(ground_truth, index=data['Book_Features'], columns=data['Book_Features']).sample(5, axis=1).sample(10, axis=0)

Melakukan evaluasi terhadap model sistem rekomendasi menggunakan metrik presisi dan recall.

In [54]:
from sklearn.metrics import precision_score, recall_score

# Mengambil sebagian kecil dari matriks cosine similarity dan matriks ground truth
sample_size = 5000
cosine_sim_sample = cosine_sim[:sample_size, :sample_size]
ground_truth_sample = ground_truth[:sample_size, :sample_size]

# Mengonversi matriks cosine similarity menjadi array satu dimensi untuk perbandingan
cosine_sim_flat = cosine_sim_sample.flatten()

# Mengonversi matriks ground truth menjadi array satu dimensi
ground_truth_flat = ground_truth_sample.flatten()

# Menghitung prediksi berdasarkan threshold
prediksi = (cosine_sim_flat >= threshold).astype(int)

# Menghitung metrik presisi dan recall
precision = precision_score(ground_truth_flat, prediksi, zero_division=1)
recall = recall_score(ground_truth_flat, prediksi, zero_division=1)

print("Precision:", precision)
print("Recall:", recall)

Presisi: 1.0
Recall: 1.0
